In [33]:
###import packages
import pandas as pd

In [34]:
###Load data from Lahman
fielding_df = pd.read_csv('lahman/fielding.csv')
player_names_df = pd.read_csv('lahman/player_info.csv', encoding='latin-1', usecols= ['playerID', 'nameFirst', 'nameLast'])

player_teams_df = pd.read_csv('lahman/batting.csv', usecols=['playerID', 'yearID', 'teamID'])

franchises_df = pd.read_csv('lahman/franchises.csv')

fielding_df

,playerID,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
0,aardsda01,2004,1,SFN,NL,P,11,0.0,32.0,0,0,0.0,0,NaN,NaN,NaN,NaN,NaN
1,aardsda01,2006,1,CHN,NL,P,45,0.0,159.0,1,5,0.0,1,NaN,NaN,NaN,NaN,NaN
2,aardsda01,2007,1,CHA,AL,P,25,0.0,97.0,2,4,1.0,0,NaN,NaN,NaN,NaN,NaN
3,aardsda01,2008,1,BOS,AL,P,47,0.0,146.0,3,6,0.0,0,NaN,NaN,NaN,NaN,NaN
4,aardsda01,2009,1,SEA,AL,P,73,0.0,214.0,2,5,0.0,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151502,zwilldu01,1915,1,CHF,FL,OF,148,NaN,NaN,356,20,8.0,6,NaN,NaN,NaN,NaN,NaN
151503,zwilldu01,1916,1,CHN,NL,OF,10,NaN,NaN,11,0,0.0,0,NaN,NaN,NaN,NaN,NaN
151504,zychto01,2015,1,SEA,AL,P,13,1.0,55.0,0,3,0.0,0,NaN,NaN,NaN,NaN,NaN
151505,zychto01,2016,1,SEA,AL,P,12,0.0,41.0,0,0,1.0,0,NaN,NaN,NaN,NaN,NaN


In [35]:
#trim down columns to only relevant ones, remove pitchers, standarize columns names
fielding_df = fielding_df.loc[fielding_df['POS'] != 'P', ['playerID', 'teamID', 'yearID', 'POS']]
fielding_df = fielding_df.rename(columns={
    'teamID': 'team',
    'yearID': 'year',
    'POS': 'position'
})

fielding_df

,playerID,team,year,position
9,aaronha01,ML1,1954,OF
10,aaronha01,ML1,1955,2B
11,aaronha01,ML1,1955,OF
12,aaronha01,ML1,1956,OF
13,aaronha01,ML1,1957,OF
...,...,...,...,...
151499,zwilldu01,CHA,1910,OF
151500,zwilldu01,CHF,1914,OF
151501,zwilldu01,CHF,1915,1B
151502,zwilldu01,CHF,1915,OF


In [36]:
player_names_df

,playerID,nameFirst,nameLast
0,aardsda01,David,Aardsma
1,aaronha01,Hank,Aaron
2,aaronto01,Tommie,Aaron
3,aasedo01,Don,Aase
4,abadan01,Andy,Abad
...,...,...,...
21005,paysojo99,Joan,Payson
21006,galbrjo99,John,Galbreath
21007,mcshejo99,John,McSherry
21008,weyerle99,Lee,Weyer


In [37]:
#merge datasets
players_df = fielding_df.merge(player_names_df, how='left', on='playerID')

#Combine names into one column
players_df['name'] = players_df['nameLast'] + ', ' + players_df['nameFirst']
players_df = players_df[['name', 'team', 'year','position']]

#Remove duplicate players
players_df = players_df.drop_duplicates()

players_df

,name,team,year,position
0,"Aaron, Hank",ML1,1954,OF
1,"Aaron, Hank",ML1,1955,2B
2,"Aaron, Hank",ML1,1955,OF
3,"Aaron, Hank",ML1,1956,OF
4,"Aaron, Hank",ML1,1957,OF
...,...,...,...,...
100133,"Zwilling, Dutch",CHA,1910,OF
100134,"Zwilling, Dutch",CHF,1914,OF
100135,"Zwilling, Dutch",CHF,1915,1B
100136,"Zwilling, Dutch",CHF,1915,OF


In [38]:
#load hitting data from Savant
hitting_df = pd.read_csv('savant/hitting-stats.csv')
hitting_df

,"last_name, first_name",player_id,year,player_age,ab,pa,hit,single,double,triple,home_run,strikeout,walk,b_gnd_into_dp,b_hit_by_pitch,b_out_fly,b_out_ground,b_out_line_drive,b_out_popup,b_reached_on_error
0,"Hunter, Torii",116338,2015,39,521,567,125,81,22,0,22,105,35,14,6,77,171,29,30,3
1,"Ortiz, David",120074,2015,39,528,614,144,70,37,0,37,95,77,16,0,77,150,48,33,5
2,"Rodriguez, Alex",121347,2015,39,523,620,131,75,22,1,33,145,84,17,6,72,141,22,25,2
3,"Ramirez, Aramis",133380,2015,37,475,516,117,68,31,1,17,68,31,23,5,85,142,48,36,1
4,"Beltré, Adrian",134181,2015,36,567,619,163,109,32,4,18,65,41,18,3,78,174,68,38,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1370,"Chourio, Jackson",694192,2024,20,528,573,145,91,29,4,21,121,39,7,3,75,141,35,14,4
1371,"Schanuel, Nolan",694384,2024,22,519,607,130,98,19,0,13,103,68,14,9,72,157,50,15,2
1372,"Langford, Wyatt",694671,2024,22,499,557,126,81,25,4,16,115,51,6,4,86,110,30,35,3
1373,"Young, Jacob",696285,2024,24,468,521,120,92,24,1,3,102,30,11,12,48,155,37,17,4


In [39]:
###merge hitting and player info datasets

#match name columns
hitting_df = hitting_df.rename(columns={'last_name, first_name': 'name'})

#merge
final_df = hitting_df.merge(players_df,'left',['name', 'year'])

#reorder columns
hitting_columns = list(hitting_df.columns)
position_columns = [col for col in final_df.columns if col not in hitting_columns]
reordered_columns = position_columns + hitting_columns
final_df = final_df[reordered_columns]

final_df

,team,position,name,player_id,year,player_age,ab,pa,hit,single,...,home_run,strikeout,walk,b_gnd_into_dp,b_hit_by_pitch,b_out_fly,b_out_ground,b_out_line_drive,b_out_popup,b_reached_on_error
0,MIN,OF,"Hunter, Torii",116338,2015,39,521,567,125,81,...,22,105,35,14,6,77,171,29,30,3
1,BOS,1B,"Ortiz, David",120074,2015,39,528,614,144,70,...,37,95,77,16,0,77,150,48,33,5
2,NYA,1B,"Rodriguez, Alex",121347,2015,39,523,620,131,75,...,33,145,84,17,6,72,141,22,25,2
3,NYA,3B,"Rodriguez, Alex",121347,2015,39,523,620,131,75,...,33,145,84,17,6,72,141,22,25,2
4,MIL,3B,"Ramirez, Aramis",133380,2015,37,475,516,117,68,...,17,68,31,23,5,85,142,48,36,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,NaN,NaN,"Chourio, Jackson",694192,2024,20,528,573,145,91,...,21,121,39,7,3,75,141,35,14,4
1996,NaN,NaN,"Schanuel, Nolan",694384,2024,22,519,607,130,98,...,13,103,68,14,9,72,157,50,15,2
1997,NaN,NaN,"Langford, Wyatt",694671,2024,22,499,557,126,81,...,16,115,51,6,4,86,110,30,35,3
1998,NaN,NaN,"Young, Jacob",696285,2024,24,468,521,120,92,...,3,102,30,11,12,48,155,37,17,4


In [40]:
### Remove duplicate players with slightly different positions (combine positions into one array)

# Group by 'player_id' and 'year', aggregating the desired columns
final_df = final_df.groupby(["player_id", "year"], as_index=False).agg({
    "position": list,  # Combine the 'position' column into a list
    **{col: "first" for col in final_df.columns if col not in ["player_id", "year", "position"]}  # Take the first value for all other columns
})

# Remove duplicates from the 'position' lists
final_df["position"] = final_df["position"].apply(lambda x: list(set(x)))

print(final_df)


      player_id  year  position  team               name  player_age   ab  \
0        116338  2015      [OF]   MIN      Hunter, Torii          39  521   
1        120074  2015      [1B]   BOS       Ortiz, David          39  528   
2        120074  2016      [1B]   BOS       Ortiz, David          40  537   
3        121347  2015  [3B, 1B]   NYA    Rodriguez, Alex          39  523   
4        133380  2015  [1B, 3B]   MIL    Ramirez, Aramis          37  475   
...         ...   ...       ...   ...                ...         ...  ...   
1370     694384  2024     [nan]  None    Schanuel, Nolan          22  519   
1371     694671  2024     [nan]  None    Langford, Wyatt          22  499   
1372     696285  2024     [nan]  None       Young, Jacob          24  468   
1373     701538  2024     [nan]  None   Merrill, Jackson          21  554   
1374     807799  2023      [OF]   BOS  Yoshida, Masataka          29  537   

       pa  hit  single  ...  home_run  strikeout  walk  b_gnd_into_dp  \
0 

In [41]:
# Print all the different stats for players
for col in final_df.columns:
    print(col)

player_id
year
position
team
name
player_age
ab
pa
hit
single
double
triple
home_run
strikeout
walk
b_gnd_into_dp
b_hit_by_pitch
b_out_fly
b_out_ground
b_out_line_drive
b_out_popup
b_reached_on_error


In [42]:
#save to csv and return dataframe
final_df.to_csv("players.csv", index=False)